# Eat Safe, Love

## Notebook Set Up

In [19]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [20]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [21]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [22]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [23]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}
results = list(establishments.find(query))

# Use count_documents to display the number of documents in the result
print("Number of establishments with hygiene score of 20:", establishments.count_documents(query))
# Display the first document in the results using pprint
pprint(results[0])

Number of establishments with hygiene score of 20: 0


In [25]:
# Convert the result to a Pandas DataFrame
hygiene_20_df = pd.DataFrame(results)
# Display the number of rows in the DataFrame
print("Rows in DataFrame:", len(hygiene_20_df))
# Display the first 10 rows of the DataFrame
hygiene_20_df.head(10)

Rows in DataFrame: 0


""


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [26]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {
    "LocalAuthorityName": {"$regex": "London", "$options": "i"},
    "RatingValue": {"$in": ["4", "5"]}
}
results = list(establishments.find(query))

# Use count_documents to display the number of documents in the result
print("Number of establishments in London with RatingValue >= 4:", establishments.count_documents(query))

# Display the first document in the results using pprint
if results:
    pprint(results[0])
else:
    print("No results found.")

Number of establishments in London with RatingValue >= 4: 0
No results found.


In [27]:
# Convert the result to a Pandas DataFrame
london_rating_df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print("Rows in DataFrame:", len(london_rating_df))

# Display the first 10 rows of the DataFrame
london_rating_df.head(10)

Rows in DataFrame: 0


""


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [28]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

query = {
    "$and": [
        {
            "$or": [
                {"RatingValue": "5"},
                {"RatingValue": 5}
            ]
        },
        {"geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search}},
        {"geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search}}
    ]
}

sort = [("scores.Hygiene", 1)]

limit = 5

# Print the results
print("Running query...")
results = list(establishments.find(query).sort(sort).limit(limit))
print("Number of matching documents:", len(results))


Running query...
Number of matching documents: 0


In [29]:
# Convert result to Pandas DataFrame
near_penang_df = pd.DataFrame(results)
near_penang_df

""


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [30]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
pipeline = [
    {"$match": {"scores.Hygiene": 0}},
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}}
]

results = list(establishments.aggregate(pipeline))


# Print the number of documents in the result
print("Number of Local Authorities with hygiene score 0:", len(results))

# Print the first 10 results
pprint(results[:10])

Number of Local Authorities with hygiene score 0: 0
[]


In [31]:
# Convert the result to a Pandas DataFrame
hygiene_zero_df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print("Rows in DataFrame:", len(hygiene_zero_df))

# Display the first 10 rows of the DataFrame
hygiene_zero_df.head(10)

Rows in DataFrame: 0


""
